In [0]:
# Databricks notebook source
# MAGIC %python
from __future__ import annotations
import json
from typing import Any, Dict, Optional
from pyspark.sql import SparkSession

In [0]:
# Exceptions (exposed globally so main/functions can use them)
class ConfigError(Exception): ...
class DataQualityError(Exception): ...

# Minimal config structure your main can read (adjust dates if you like)
cfg = {
    "api": {
        "base_url": "https://api.nasa.gov/neo/rest/v1/feed",
        "start_date": "2015-09-07",
        "end_date": "2015-09-08",
        "api_key": "DEMO_KEY"
    },
    "paths": {
        # CE-safe locations (do NOT use /mnt)
        "raw_dir": "dbfs:/FileStore/raw/nasa/neo",
        # a single-file cache for the API response (Spark will read it back)
        "raw_api_cache": "dbfs:/FileStore/raw/nasa/neo/neo_{start}_{end}.json"
    },
    "tables": {
        "neo_objects": "default.neo_objects_raw",          # all NEOs
        "close_approaches": "default.neo_close_approaches" # all close approach instances
    }
}

# Optional: convenience to pretty-print config in logs
def show_config(d):
    import json
    return json.dumps(d, indent=2, sort_keys=True)
